In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import librosa
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def pre_emphasis(audio, pre_emphasis_coeff=0.97):
    return np.append(audio[0], audio[1:] - pre_emphasis_coeff * audio[:-1])

# Fungsi untuk ekstraksi MFCC dari file audio
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, sr=None)
        print(f'sample rate: {sample_rate}')
        if len(audio) > 4 * sample_rate:
            audio = audio[:4 * sample_rate]
        # Pad audio to 4 seconds if shorter than 4 seconds
        elif len(audio) < 4 * sample_rate:
            padding = 4 * sample_rate - len(audio)
            audio = np.pad(audio, (0, padding), 'constant')
        # Apply pre-emphasis filter
        audio = pre_emphasis(audio)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)
        # mfccs_mean = np.mean(mfccs.T, axis=0)
        # print(f'mfccs_mean: {mfccs_mean}')
        print(f'mfccs: {mfccs.shape}')
    except Exception as error:
        print(f"Error encountered while parsing file: {file_name}")
        print(f'error : {error}')
        return None
    return mfccs

In [ ]:
# Direktori dataset dan daftar emosi
dataset_path = "/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24"  # ganti dengan path ke dataset Anda
emotions = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']
files = os.listdir(dataset_path)
print(len(files))

24


In [ ]:
# Daftar emosi berdasarkan nomor pada file RAVDESS
emotion_labels = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

In [ ]:
# List untuk menyimpan fitur dan label
features = []
labels = []

In [ ]:
# Loop melalui file audio dalam dataset
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(".wav"):
            file_path = os.path.join(root, file)
            print(file_path)
            mfccs = extract_features(file_path)
            if mfccs is not None:
                features.append(mfccs)
                # Ekstraksi label emosi dari nama file RAVDESS
                label = emotion_labels[file.split("-")[2]]
                labels.append(label)

/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-01-01-01-01-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-02-01-01-02-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-01-01-01-02-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-02-02-01-01-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-02-01-02-01-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-01-01-02-01-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_actors_01-24/Actor_23/03-01-01-01-02-02-23.wav
sample rate: 48000
mfccs: (13, 376)
/content/drive/MyDrive/Colab Notebooks/audio_speech_act

In [ ]:
# Konversi ke numpy array
X = np.array(features)
y = np.array(labels)

In [ ]:
# Encode label menjadi angka
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
# Bagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
                                  # k-fold crosss valition

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1152, 13)
(288, 13)
(1152,)
(288,)


In [ ]:
classifier = RandomForestClassifier() # inisiasi model

classifier.fit(X_train, y_train) # proses training

y_pred = classifier.predict(X_test) # testing

accuracy = accuracy_score(y_test, y_pred) # evaluasi
print(accuracy)

0.5763888888888888


In [ ]:
import pickle
with open('classifier.pkl', 'wb') as file:
    pickle.dump(classifier, file)

In [ ]:
classification_result = classification_report(y_test, y_pred)
print(classification_result)

              precision    recall  f1-score   support

           0       0.53      0.83      0.65        30
           1       0.61      0.85      0.71        48
           2       0.76      0.54      0.63        41
           3       0.55      0.49      0.52        35
           4       0.68      0.57      0.62        40
           5       0.45      0.24      0.31        21
           6       0.45      0.41      0.43        32
           7       0.55      0.53      0.54        43

    accuracy                           0.58       290
   macro avg       0.57      0.56      0.55       290
weighted avg       0.59      0.58      0.57       290



In [ ]:
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
svm_classifier = SVC(kernel='rbf')

svm_classifier.fit(X_train, y_train) # proses training

y_pred_svm = svm_classifier.predict(X_test) # testing

accuracy = accuracy_score(y_test, y_pred_svm) # evaluasi
print(accuracy)

0.29310344827586204


In [ ]:
classification_result = classification_report(y_test, y_pred_svm)
print(classification_result)

              precision    recall  f1-score   support

           0       0.29      0.67      0.41        30
           1       0.42      0.75      0.54        48
           2       0.00      0.00      0.00        41
           3       0.00      0.00      0.00        35
           4       0.14      0.07      0.10        40
           5       0.00      0.00      0.00        21
           6       0.21      0.28      0.24        32
           7       0.26      0.40      0.31        43

    accuracy                           0.29       290
   macro avg       0.16      0.27      0.20       290
weighted avg       0.18      0.29      0.22       290



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=5)

knn_clf.fit(X_train, y_train) # proses training

y_pred_knn = knn_clf.predict(X_test) # testing

accuracy = accuracy_score(y_test, y_pred_knn) # evaluasi
print(accuracy)

0.33793103448275863


In [ ]:
classification_result = classification_report(y_test, y_pred_knn)
print(classification_result)

              precision    recall  f1-score   support

           0       0.28      0.63      0.38        30
           1       0.48      0.62      0.54        48
           2       0.30      0.22      0.25        41
           3       0.39      0.34      0.36        35
           4       0.43      0.33      0.37        40
           5       0.18      0.14      0.16        21
           6       0.15      0.12      0.14        32
           7       0.35      0.19      0.24        43

    accuracy                           0.34       290
   macro avg       0.32      0.32      0.31       290
weighted avg       0.34      0.34      0.32       290



In [ ]:
from sklearn.model_selection import cross_validate, cross_val_predict

In [ ]:
# buat modelnya
randomForest = RandomForestClassifier()
svm = SVC()

scoring = ['accuracy', 'precision_macro', 'recall_macro']

scores = cross_validate(randomForest, X, y_encoded, scoring=scoring, cv=10) # 10-fold validation
y_pred = cross_val_predict(randomForest, X, y_encoded, cv=5)

In [ ]:
clf_result = classification_report(y_encoded, y_pred)

print(clf_result)

              precision    recall  f1-score   support

           0       0.55      0.63      0.59       192
           1       0.40      0.58      0.47       192
           2       0.31      0.33      0.32       192
           3       0.41      0.34      0.38       192
           4       0.43      0.37      0.40       193
           5       0.23      0.07      0.11        96
           6       0.35      0.32      0.33       192
           7       0.25      0.27      0.26       197

    accuracy                           0.38      1446
   macro avg       0.37      0.36      0.36      1446
weighted avg       0.38      0.38      0.37      1446



In [ ]:
gauss_classifier = GaussianProcessClassifier(1.0 * RBF(1.0))

gauss_classifier.fit(X_train, y_train) # proses training

y_pred_gauss = gauss_classifier.predict(X_test) # testing

accuracy = accuracy_score(y_test, y_pred_gauss) # evaluasi
print(accuracy)

KeyboardInterrupt: 

In [ ]:
classification_result = classification_report(y_test, y_pred_gauss)
print(classification_result)